In [5]:
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dropout
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
import re
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.utils import resample



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Načítanie dát
dawid = pd.read_csv('labeled_data.csv')

# Odstránenie prvého stĺpca
dawid = dawid.drop(dawid.columns[0], axis=1)

# Prekódovanie tried

dawid.loc[dawid['class'] == 0, 'class'] = 1
dawid.loc[dawid['class'] == 2, 'class'] = 0

# Odstránenie nepotrebných stĺpcov
dawid = dawid.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'])

print(dawid.head(100))

# Extrakcia textov a tried do samostatných premenných
texts = dawid['tweet']
labels = dawid['class']


    class                                              tweet
0       0  !!! RT @mayasolovely: As a woman you shouldn't...
1       1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2       1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3       1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4       1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
..    ...                                                ...
95      1  "@CauseWereGuys: Going back to school sucks mo...
96      1  "@CauseWereGuys: On my way to fuck yo bitch ht...
97      1  "@CeleyNichole: @white_thunduh how come you ne...
98      1  "@ChadMFVerbeck: If Richnow doesn't show up wi...
99      1  "@ChandlerParsons: How bout them Cowboys!!!!" ...

[100 rows x 2 columns]


In [6]:
# X obsahuje texty komentárov a y - ich triedy

# Rozdelenie na trénovaciu a testovaciu sadu (80:20)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Vytvorenie train dat
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv('trainNO.csv', index=False)

# Rozdelenie train dát na toxické a netoxické komentáre
toxic_comments = train_data[train_data['class'] == 1]
non_toxic_comments = train_data[train_data['class'] == 0]

# Zníženie počtu toxických komentárov na rovnaké množstvo ako netoxické
toxic_comments_downsampled = resample(toxic_comments,
                                     replace=False,  # Nezameníme prvky
                                     n_samples=len(non_toxic_comments),  # Nastavoovanie počtu na rovnaké množstvo ako netoxické komentáre
                                     random_state=42)

# Spojenie zníženého množstva toxických komentárov s netoxickými
balanced_train_data = pd.concat([toxic_comments_downsampled, non_toxic_comments])

# Premiešanie dát
balanced_train_data = balanced_train_data.sample(frac=1, random_state=42)


X_train_balanced = balanced_train_data.drop('class', axis=1)
y_train_balanced = balanced_train_data['class']

# Spojenie do jedného datasetu pre tréning a test
train = pd.concat([X_train_balanced, y_train_balanced], axis=1)
test = pd.concat([X_test, y_test], axis=1)

# Prevedenie tried na one-hot encoding
one_hot_labels = to_categorical(labels, num_classes=2)

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
